In [2]:
import maliput

In [3]:
help(maliput.api)

Help on module maliput.api in maliput:

NAME
    maliput.api

CLASSES
    pybind11_builtins.pybind11_object(builtins.object)
        InertialPosition
        Junction
        JunctionId
        Lane
        LaneId
        LanePosition
        LanePositionResult
        LaneSRange
        LaneSRoute
        RoadGeometry
        RoadGeometry.IdIndex
        RoadGeometryId
        RoadNetwork
        RoadPosition
        RoadPositionResult
        Rotation
        SRange
        Segment
        SegmentId
    
    class InertialPosition(pybind11_builtins.pybind11_object)
     |  Method resolution order:
     |      InertialPosition
     |      pybind11_builtins.pybind11_object
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  Distance(...)
     |      Distance(self: maliput.api.InertialPosition, inertial_position: maliput.api.InertialPosition) -> float
     |  
     |  __eq__(...)
     |      __eq__(self: maliput.api.InertialPosition, arg0: maliput.api.Ine

In [4]:
help(maliput.math)

Help on module maliput.math in maliput:

NAME
    maliput.math

CLASSES
    pybind11_builtins.pybind11_object(builtins.object)
        Quaternion
        RollPitchYaw
        Vector3
        Vector4
    
    class Quaternion(pybind11_builtins.pybind11_object)
     |  Method resolution order:
     |      Quaternion
     |      pybind11_builtins.pybind11_object
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(...)
     |      __init__(self: maliput.math.Quaternion, arg0: float, arg1: float, arg2: float, arg3: float) -> None
     |  
     |  __str__(...)
     |      __str__(self: maliput.math.Quaternion) -> str
     |  
     |  coeffs(...)
     |      coeffs(self: maliput.math.Quaternion) -> maliput.math.Vector4
     |  
     |  w(...)
     |      w(self: maliput.math.Quaternion) -> float
     |  
     |  x(...)
     |      x(self: maliput.math.Quaternion) -> float
     |  
     |  y(...)
     |      y(self: maliput.math.Quaternion) -> float
   

In [5]:
manager = maliput.plugin.MaliputPluginManager()
manager.ListPlugins()


['maliput_dragway', 'maliput_multilane', 'maliput_malidrive']

In [6]:
# https://shiny-fiesta-6b790eb7.pages.github.io/from_doxygen/html/deps/maliput_malidrive/html/group__road__network__configuration__builder__keys.html
# 
road_network_options = {
    "opendrive_file": "../../maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.xodr"
}


In [7]:
network1 = maliput.plugin.create_road_network_from_plugin("maliput_malidrive", road_network_options)

In [8]:
help(network1.road_geometry())
help(network1.road_geometry().ById())
help(network1.road_geometry().junction(0))
help(network1.road_geometry().junction(0).segment(0))
help(network1.road_geometry().junction(0).segment(0).lane(0))
help(network1.road_geometry().junction(0).segment(0).lane(0).id())
help(maliput.api.SRange)
help(maliput.api.LaneSRange)
geometry = network1.road_geometry()

Help on RoadGeometry in module maliput.api object:

class RoadGeometry(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      RoadGeometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  ById(...)
 |      ById(self: maliput.api.RoadGeometry) -> maliput::api::RoadGeometry::IdIndex
 |  
 |  FindRoadPositions(...)
 |      FindRoadPositions(self: maliput.api.RoadGeometry, inertial_position: maliput.api.InertialPosition, radius: float) -> List[maliput.api.RoadPositionResult]
 |  
 |  ToRoadPosition(...)
 |      ToRoadPosition(self: maliput.api.RoadGeometry, inertial_position: maliput.api.InertialPosition) -> maliput.api.RoadPositionResult
 |  
 |  ToRoadPositionByHint(...)
 |      ToRoadPositionByHint(self: maliput.api.RoadGeometry, inertial_position: maliput.api.InertialPosition, hint: maliput.api.RoadPosition) -> maliput.api.RoadPositionResult
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  Se

In [10]:
# Iterate all items.
print("Geometry. Num junctions: {}".format(geometry.num_junctions()))
for junction_index in range(geometry.num_junctions()):
    junction = geometry.junction(junction_index)
    # help(junction)
    print("  Junction {}: Segments: {}".format(junction.id().string(), junction.num_segments()))
    for segment_index in range(junction.num_segments()):
        segment = junction.segment(segment_index)
        print("    Segment {}: Lanes: {}".format(segment.id().string(), segment.num_lanes()))
        for lane_index in range(segment.num_lanes()):
            lane = segment.lane(lane_index)
            print("    Lane {}: length: {}".format(lane.id().string(), lane.length()))
            

Geometry. Num junctions: 11
  Junction 0_0: Segments: 1
    Segment 0_0: Lanes: 2
    Lane 0_0_-1: length: 174.77007946851094
    Lane 0_0_1: length: 169.27190949567478
  Junction 1_0: Segments: 1
    Segment 1_0: Lanes: 2
    Lane 1_0_-1: length: 90.20944440138805
    Lane 1_0_1: length: 90.23055559858
  Junction 10: Segments: 84
    Segment 18_0: Lanes: 1
    Lane 18_0_1: length: 0.1799999999981746
    Segment 18_1: Lanes: 1
    Lane 18_1_1: length: 2.100000000000009
    Segment 18_2: Lanes: 1
    Lane 18_2_1: length: 0.05000000000001136
    Segment 18_3: Lanes: 1
    Lane 18_3_1: length: 13.529999999999967
    Segment 18_4: Lanes: 1
    Lane 18_4_1: length: 0.05000000000001136
    Segment 18_5: Lanes: 1
    Lane 18_5_1: length: 2.090000000000001
    Segment 18_6: Lanes: 1
    Lane 18_6_1: length: 0.17999999999818642
    Segment 19_0: Lanes: 1
    Lane 19_0_-1: length: 0.1799999999981746
    Segment 19_1: Lanes: 1
    Lane 19_1_-1: length: 2.100000000000009
    Segment 19_2: Lanes: 1

In [11]:
# Sample so I can get help.
p_inertial = maliput.api.InertialPosition(0,0,0)
road_position_result = geometry.ToRoadPosition(p_inertial)
road_position = road_position_result.road_position
help(p_inertial)
help(road_position_result)
help(road_position)
help(road_position.lane)
help(road_position.pos)


Help on InertialPosition in module maliput.api object:

class InertialPosition(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      InertialPosition
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Distance(...)
 |      Distance(self: maliput.api.InertialPosition, inertial_position: maliput.api.InertialPosition) -> float
 |  
 |  __eq__(...)
 |      __eq__(self: maliput.api.InertialPosition, arg0: maliput.api.InertialPosition) -> bool
 |  
 |  __init__(...)
 |      __init__(self: maliput.api.InertialPosition, x: float, y: float, z: float) -> None
 |  
 |  length(...)
 |      length(self: maliput.api.InertialPosition) -> float
 |  
 |  set_x(...)
 |      set_x(self: maliput.api.InertialPosition, arg0: float) -> None
 |  
 |  set_y(...)
 |      set_y(self: maliput.api.InertialPosition, arg0: float) -> None
 |  
 |  set_z(...)
 |      set_z(self: maliput.api.InertialPosition, arg0: float) -> None
 |  
 |  x(...)
 

In [12]:
# Perform some to road queries, to lane queries, and back.
positions = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 25, 0]
]
for position in positions:
    p_inertial = maliput.api.InertialPosition(*position)
    # Would be nice to have __repr__
    print("inertial {}, {}, {}".format(p_inertial.x(), p_inertial.y(), p_inertial.z()))
    road_position_result = geometry.ToRoadPosition(p_inertial)
    road_position = road_position_result.road_position
    print(
        "road position {}, ({},{},{})".format(
            str(road_position.lane.id().string()),
            road_position.pos.s(),
            road_position.pos.r(),
            road_position.pos.h()
        )
    )
    # back to inertial
    p_roundtrip = road_position.lane.ToInertialPosition(road_position.pos)
    print("inertial roundtrip {}, {}, {}".format(p_roundtrip.x(), p_roundtrip.y(), p_roundtrip.z()))
    
    
    print()

inertial 0.0, 0.0, 0.0
road position 96_3_-1, (6.342075680086242,1.2679070179605731,0.0)
inertial roundtrip 1.6826817716975029e-15, -1.1102230246251565e-16, 0.0

inertial 0.0, 1.0, 0.0
road position 96_3_-1, (6.350642120338619,0.267943710583046,0.0)
inertial roundtrip -9.679756995950584e-16, 1.0, 0.0

inertial 0.0, 25.0, 0.0
road position 2_0_-1, (16.39078594979164,1.2830799963633912,0.0)
inertial roundtrip 5.551115123125783e-17, 25.000000000000004, 0.0



In [17]:
the_range = maliput.api.SRange(0,1)
the_lane_range = maliput.api.LaneSRange(road_position.lane.id(), the_range)
the_range_2 = maliput.api.SRange(1,0.5)
arbitrary_lane_id = network1.road_geometry().junction(0).segment(0).lane(0).id()
the_lane_range_2 = maliput.api.LaneSRange(arbitrary_lane_id, the_range_2)

In [30]:
print("range", the_range.s0(), the_range.s1(), the_range.size(), "with_s", the_range.WithS())
print("range2", the_range_2.s0(), the_range_2.s1(), the_range_2.size(), "with_s", the_range_2.WithS())

range 0.0 1.0 1.0 with_s 1.0
range2 1.0 0.5 0.5 with_s 0.0


In [37]:
overlap1 = the_range.GetIntersection(the_range_2, 0.01)
overlap2 = the_range_2.GetIntersection(the_range, 0.01)
# Order not preserved. Okay with me!
print("intersects", the_range.Intersects(the_range_2, 0.01), "overlap", overlap1.s0(), overlap1.s1(), overlap1.size())
print("intersects", the_range_2.Intersects(the_range, 0.01), "overlap", overlap2.s0(), overlap2.s1(), overlap2.size())
print("lane intersects?", the_lane_range.Intersects(the_lane_range_2, 0.01))

intersects True overlap 0.5 1.0 0.5
intersects True overlap 0.5 1.0 0.5
lane intersects? False


In [38]:
# Now load a road network with rulebooks.
# https://shiny-fiesta-6b790eb7.pages.github.io/from_doxygen/html/deps/maliput_malidrive/html/group__road__network__configuration__builder__keys.html
# 
road_network_options2 = {
    "opendrive_file": "../../maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.xodr",
    "road_rule_book": "../../maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.yaml",
}
road_network_options2["traffic_light_book"] = road_network_options2["road_rule_book"]
road_network_options2["phase_ring_book"] = road_network_options2["road_rule_book"]
road_network_options2["intersection_book"] = road_network_options2["road_rule_book"]


In [39]:
network2 = maliput.plugin.create_road_network_from_plugin("maliput_malidrive", road_network_options2)

In [40]:
help(network2)

Help on RoadNetwork in module maliput.api object:

class RoadNetwork(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      RoadNetwork
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  road_geometry(...)
 |      road_geometry(self: maliput.api.RoadNetwork) -> maliput::api::RoadGeometry
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_builtins.pybind11_object:
 |  
 |  __new__(*args, **kwargs) from pybind11_builtins.pybind11_type
 |      Create and return a new object.  See help(type) for accurate signature.

